In [1]:
from platform import python_version
# !pip install shap==0.39
# !pip install transformers==4.5
python_version()

'3.7.13'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from data.loader import dataset_loader, DatasetEnum
import shap

# Load model from huggingface

In [4]:
tokenizer = AutoTokenizer.from_pretrained("ghanashyamvtatti/roberta-fake-news")
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news").cpu()
pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, binary_output=True)

# Load the respective dataset for the model

In [5]:
true_dataset, fake_dataset = dataset_loader(DatasetEnum.ROBERTA_FAKE_NEWS)

Loading true instances from dir: /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/RobertaFakeNews/True.csv
Loading fake instances from dir: /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/RobertaFakeNews/Fake.csv


In [6]:
def explain_text(text):
    print(f'Explaining the following text: \n---------------------------------------------------------\n{text}\n--------------------------------------------------------')
    prediction = pipeline([text])
    print(f'Prediction: {prediction}')
    explainer = shap.Explainer(pipeline, output_names=["fake", "true"])
    shap_values = explainer([text])
    shap.text_plot(shap_values)
    return shap_values

In [ ]:
# we get the first 512 chars since the model accepts that many
# get first 10 samples for experimenting
true_samples = true_dataset["text"].map(lambda x: x[:512])[:10]
fake_samples = fake_dataset["text"].map(lambda x: x[:512])[:10]

In [ ]:
true_sample_shap_values = explain_text(true_samples[0])

In [ ]:
fake_sample_shap_values = explain_text(fake_samples[0])